<a id="table_of_content"></a>

# Part 1: Bag-of-Words Exercises

In the following, we will convert our corpus of 10-k documents into bag-of-words, and convert them into document vectors using Term-Frequency-Inverse-Document-Frequency (tf-idf) re-weighting. Afterward, we will compute sentiments and similarity metrics. Since we will be reusing our notebook, so the various sections are linked below:

1. <a href="#bag_of_word">Compute bag-of-words </a>: implement `bag_of_words` that converts tokenized words into a dictionary of word-counts

2. <a href="#sentiment">Sentiments </a>: using wordlists, compute positive and negative sentiments from bag-of-words. Implement `get_sentiment`

For solutions, see [bagofwords_solutions.py](./bagofwords_solutions.py). You can load the functions by simply calling 

`from bagofwords_solutions import *`

# Part 2: Document-Vector Exercises

3. <a href="#compute_idf">Compute idf </a>: computing the inverse document frequency, implement `get_idf`

4. <a href="#compute_tf">Compute tf </a>: computing the term frequency, implement `get_tf`

5. <a href="#doc_vector">Document vector </a>: using the functions `get_idf` and `get_tf`, compute a word_vector for each document in the corpus
6. <a href="#similarity">Similarities </a>: comparing two vectors, and compute cosine and jacard similarity metrics. Implement `get_cos` and `get_jac`

For solutions, see [bagofwords_solutions.py](./bagofwords_solutions.py). You can load the functions by simply calling 

`from bagofwords_solutions import *`


## 0. Initialization

First we read in our 10-k documents

In [1]:
# download some excerpts from 10-K files
from download10k import get_files

CIK = {'ebay': '0001065088', 'apple':'0000320193', 'sears': '0001310067'}
get_files(CIK['ebay'], 'EBAY')
get_files(CIK['apple'], 'AAPL')
get_files(CIK['sears'], 'SHLDQ')


# get a list of all 10-ks in our directory
files=! ls *10k*.txt
print("10-k files: ",files)
files = [open(f,"r").read() for f in files]

downloading 10-Ks item 1A for CIK = 0001065088 ...
downloading 10-Ks item 1A for CIK = 0000320193 ...
downloading 10-Ks item 1A for CIK = 0001310067 ...
skipping SHLDQ_10k_2017.txt
skipping SHLDQ_10k_2016.txt
skipping SHLDQ_10k_2015.txt
skipping SHLDQ_10k_2014.txt
skipping SHLDQ_10k_2013.txt
10-k files:  ['AAPL_10k_2013.txt', 'AAPL_10k_2014.txt', 'AAPL_10k_2015.txt', 'AAPL_10k_2016.txt', 'AAPL_10k_2017.txt', 'EBAY_10k_2013.txt', 'EBAY_10k_2014.txt', 'EBAY_10k_2015.txt', 'EBAY_10k_2016.txt', 'EBAY_10k_2017.txt', 'SHLDQ_10k_2013.txt', 'SHLDQ_10k_2014.txt', 'SHLDQ_10k_2015.txt', 'SHLDQ_10k_2016.txt', 'SHLDQ_10k_2017.txt']


here we define useful functions to tokenize the texts into words, remove stop-words, and lemmatize and stem our words

In [2]:
import numpy as np

# for nice number printing
np.set_printoptions(precision=3, suppress=True)

# tokenize and clean the text
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from collections import Counter
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
# tokenize anything that is not a number and not a symbol
word_tokenizer = RegexpTokenizer(r'[^\d\W]+')

nltk.download('stopwords')
nltk.download('wordnet')


sno = SnowballStemmer('english')
wnl = WordNetLemmatizer()

# get our list of stop_words
nltk.download('stopwords')
stop_words = set(stopwords.words('english')) 
# add some extra stopwords
stop_words |= {"may", "business", "company", "could", "service", "result", "product", 
               "operation", "include", "law", "tax", "change", "financial", "require",
               "cost", "market", "also", "user", "plan", "actual", "cash", "other",
               "thereto", "thereof", "therefore"}

# useful function to print a dictionary sorted by value (largest first by default)
def print_sorted(d, ascending=False):
    factor = 1 if ascending else -1
    sorted_list = sorted(d.items(), key=lambda v: factor*v[1])
    for i, v in sorted_list:
        print("{}: {:.3f}".format(i, v))

# convert text into bag-of-words
def clean_text(txt):
    lemm_txt = [ wnl.lemmatize(wnl.lemmatize(w.lower(),'n'),'v') \
                for w in word_tokenizer.tokenize(txt) if \
                w.isalpha() and w not in stop_words ]
    return [ sno.stem(w) for w in lemm_txt if w not in stop_words and len(w) > 2 ]

corpus = [clean_text(f) for f in files]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<a id="bag_of_words"></a>

## 1. Bag-of-Words

Implement a function that converts a list of tokenized words into bag-of-words, i.e. a dictionary that outputs the frequency count of a word

** python already provide the `collections.Counter` class to perform this, but I encourage you to implement your own function as an exercise

<a href="#table_of_content">back to top</a>

In [3]:
from collections import defaultdict

def bag_of_words(words):
    bag = defaultdict(int)
    for w in words:
        bag[w] += 1
    return bag

<a id="sentiment"></a>

## 2. Sentiments
Count the fraction of words that appear in a wordlist, returning a sentiment score between 0 and 1:

$$
\textrm{score} = \frac{\textrm{Number of words in document matching wordlist}}{\textrm{Number of words in document}}
$$

Implement the score in a function `get_sentiment(words, wordlist)`, where words is a list of words. Feel free to use the previous `bag_of_words` function. 
(*for extra challenge, try to code the function in one-line*)

Here, I've included a positive and negative wordlist that I constructed by hand. Due to copyright issues, we are not able to provide other commonly used wordlists. I encourage you to try out different wordlists on your own.

<a href="#table_of_content">back to top</a>

In [10]:
# load wordlist first
import pickle

with open('positive_words.pickle', 'rb') as f:
    positive_words = pickle.load(f)
    # also need to stem and lemmatize the text
    positive_words = set(clean_text(" ".join(positive_words)))
    
with open('negative_words.pickle', 'rb') as f:
    negative_words = pickle.load(f)
    negative_words = set(clean_text(" ".join(negative_words)))
    
# check out the list
print("positive words: ", positive_words)
print("negative words: ", negative_words)

positive words:  {'achiev', 'posit', 'forefront', 'perfect', 'esteem', 'wealth', 'strong', 'stabl', 'courag', 'best', 'fresh', 'exemplifi', 'victori', 'revolution', 'brilliant', 'magic', 'prestig', 'better', 'earnest', 'special', 'prestigi', 'appeal', 'great', 'invent', 'extraordinarili', 'reliev', 'aptitud', 'creativ', 'resourc', 'virtuous', 'happi', 'devot', 'clean', 'nobl', 'clear', 'monument', 'dynam', 'extraordinari', 'solv', 'groundbreak', 'generous', 'fantast', 'product', 'meritori', 'fortun', 'buoy', 'boon', 'benevol', 'fabul', 'award', 'famous', 'upbeat', 'trendi', 'favor', 'profici', 'independ', 'win', 'wonder', 'success', 'love', 'impress', 'miracl', 'flourish', 'growth', 'harmoni', 'winner', 'advanc', 'progress', 'congratul', 'nobli', 'solut', 'heal', 'loyalti', 'spring', 'optimist', 'ador', 'rebound', 'divin', 'satisfi', 'stabil', 'restor', 'attract', 'trendili', 'terrif', 'nobil', 'benign', 'legendari', 'improv', 'awesom', 'jubil', 'amaz', 'wholesom', 'honor', 'lucrat', '

In [8]:
def get_sentiment(txt, wordlist):
    matching_words = [w for w in txt if w in wordlist]
    return len(matching_words)/len(txt)

In [9]:
# test your function
positive_sentiments = np.array([ get_sentiment(c, positive_words) for c in corpus ])
print(positive_sentiments)

negative_sentiments = np.array([ get_sentiment(c, negative_words) for c in corpus ])
print(negative_sentiments)

[ 0.034  0.034  0.033  0.032  0.032  0.036  0.034  0.034  0.034  0.031
  0.037  0.033  0.037  0.039  0.04 ]
[ 0.06   0.054  0.053  0.052  0.052  0.05   0.05   0.051  0.054  0.061
  0.06   0.059  0.058  0.054  0.059]


**before continuing part 2, go through the lesson material first!**

<a id="compute_idf"></a>

# Part 2 Document Vector Exercises

## 3. Computer idf
Given a corpus, or a list of bag-of-words, we want to compute for each word $w$, the inverse-document-frequency, or ${\rm idf}(w)$. This can be done in a few steps:

1. Gather a set of all the words in all the bag-of-words (python set comes in handy, and the union operator `|` is useful here)


2. Loop over each word $w$, and compute ${\rm df}_w$, the number of documents where this word appears at least once. A dictionary is useful for keeping track of ${\rm df}_w$


3. After computing ${\rm df}_w$, we can compute ${\rm idf}(w)$. There are usually two possibilities, the simplest one is 
$${\rm idf}(w)=\frac{N}{{\rm df}_w}$$
Where $N$ is the total number of documents in the corpus and $df_w$ the number of documents that contain the word $w$. Frequently, a logarithm term is added as well
$${\rm idf}(w)=\log\frac{N}{{\rm df}_w}$$
One issue with using the logarithm is that when ${\rm df}_w = N$, ${\rm idf}(w)=0$, indicating that words common to all documents would be ignored. If we don't want this behavior, we can define ${\rm idf}(w)=\log\left(1+N/{\rm df}_w\right)$ or ${\rm idf}(w)=1+\log\left(N/{\rm df}_w\right)$ instead. For us, we'll not use the extra +1 for ${\rm idf}$.

In the following, define a function called `get_idf(corpus, include_log=True)` that computes ${\rm idf}(w)$ for all the words in a corpus, where `corpus` for us is a processed list of bag-of-words (stemmed and lemmatized). The optional parameter `include_log` includes the logarithm in the computation.

<a href="#table_of_content">back to top</a>

In [19]:
# compute idf
def get_idf(corpus, include_log=True):
    N = len(corpus)
    freq = defaultdict(int)
    words = set()
    for c in corpus:
        words |= set(c)
    for w in words:
        freq[w] = sum([w in c for c in corpus])
    if include_log:
        return {w : np.log(N/freq[w]) for w in freq}
    else:
        return {w : N/freq[w] for w in freq}


You should expect to see many idf values = 0! This is by design, because we have ${\rm idf}(w)=\log N_d/{\rm df}_w$ and $N_d/{\rm df}_w=1$ for the most common words!

In [20]:
# test your code
idf=get_idf(corpus)
print_sorted(idf, ascending=True)

duti: 0.000
brand: 0.000
volatil: 0.000
inher: 0.000
base: 0.000
face: 0.000
onlin: 0.000
must: 0.000
outsourc: 0.000
harm: 0.000
fail: 0.000
reason: 0.000
liabil: 0.000
margin: 0.000
contract: 0.000
militari: 0.000
regard: 0.000
rate: 0.000
initi: 0.000
strategi: 0.000
within: 0.000
complex: 0.000
qualiti: 0.000
delay: 0.000
maintain: 0.000
continu: 0.000
abl: 0.000
trade: 0.000
abil: 0.000
similar: 0.000
virus: 0.000
factor: 0.000
area: 0.000
local: 0.000
outstand: 0.000
prospect: 0.000
power: 0.000
activ: 0.000
exchang: 0.000
decreas: 0.000
natur: 0.000
order: 0.000
use: 0.000
impact: 0.000
long: 0.000
risk: 0.000
unabl: 0.000
consolid: 0.000
challeng: 0.000
limit: 0.000
credit: 0.000
declin: 0.000
breach: 0.000
attract: 0.000
industri: 0.000
unfavor: 0.000
acquir: 0.000
softwar: 0.000
agreement: 0.000
rise: 0.000
expans: 0.000
debt: 0.000
conflict: 0.000
relat: 0.000
purchas: 0.000
loss: 0.000
common: 0.000
accord: 0.000
thus: 0.000
number: 0.000
circumst: 0.000
across: 0.000
incre

<a id="compute_tf"></a>

## 4. Compute tf

Below we will compute ${\rm tf}(w,d)$, or the term frequency for a given word $w$, in a given document $d$. Since our ultimate goal is to compute a document vector, we'd like to keep a few things in mind

1. Store the ${\rm tf}(w, d)$ for each word in a document as a dictionary
2. Even when a word doesn't appear in the document $d$, we still want to keep a $0$ entry in the dictionary. This is important when we convert the dictionary to a vector, where zero entries are important


There are multiple definitions for ${\rm tf}(w,d)$, the simplest one is

$$
{\rm tf}(w,d)=\frac{f_{w,d}}{a_d}
$$

where $f_{w,d}$ is the number of occurence of the word $w$ in the document $d$, and $a$ the average occurence of all the words in that document for normalization. Just like ${\rm idf}(w)$, a logarithm can be added

$$
{\rm tf}(w,d)=\begin{cases}
\frac{1+\log f_{w,d}}{1+\log a_d} & f_{w,d} > 0 \\
0 & f_{w,d} = 0 \\
\end{cases}
$$

Implement the function `get_df(txt, include_log=True)` that computes ${\rm tf}(w,d)$ for each word in the document (returns a defaultdict(int), so that when supplying words not in the document the dictionary will yield zero instead of an error). Also include the optional parameter `include_log` that enables the additional logarithm term in the computation. I suggest also adding a function called `_tf` that computes the function above. 

<a href="#table_of_content">back to top</a>

In [21]:
import numpy as np
from math import *

def _tf(freq, avg, include_log=True):
    if include_log:
        return 0 if freq == 0 else (1+np.log(freq))/(1+np.log(avg))
    else:
        return freq/avg

def get_tf(txt, include_log=True):
    freq = bag_of_words(txt)
    avg = np.mean(list(freq.values()))
    tf = {w : _tf(f, avg, include_log) for w,f in freq.items()}
    return defaultdict(int, tf)

In [22]:
tfs = [ get_tf(c) for c in corpus ]
print_sorted(tfs[0])

affect: 2.105
oper: 2.066
materi: 2.041
condit: 2.033
advers: 2.024
compon: 1.901
signific: 1.876
risk: 1.850
develop: 1.837
price: 1.823
parti: 1.793
new: 1.777
custom: 1.777
subject: 1.777
third: 1.777
continu: 1.726
increas: 1.707
system: 1.667
technolog: 1.667
intern: 1.667
foreign: 1.667
effect: 1.646
softwar: 1.646
applic: 1.646
manag: 1.646
demand: 1.623
substanti: 1.623
limit: 1.623
credit: 1.600
addit: 1.600
partner: 1.600
manufactur: 1.600
futur: 1.600
compet: 1.600
valu: 1.574
sale: 1.574
competit: 1.574
relat: 1.574
mani: 1.574
devic: 1.574
depend: 1.547
secur: 1.547
invest: 1.547
margin: 1.547
time: 1.547
distribut: 1.547
avail: 1.547
retail: 1.547
currenc: 1.519
supplier: 1.519
obtain: 1.519
content: 1.519
provid: 1.519
inform: 1.519
store: 1.519
resel: 1.519
regul: 1.519
industri: 1.488
failur: 1.488
experi: 1.488
use: 1.488
assur: 1.488
econom: 1.454
sell: 1.454
purchas: 1.454
loss: 1.454
term: 1.454
licens: 1.454
trade: 1.454
channel: 1.418
competitor: 1.418
certain: 1

<a id="doc_vector"></a>

## 5. Document Vector
Combine the implementation for ${\rm tf}(w,d)$ and ${\rm idf}(w)$ to compute a word-vector for each document in a corpus. Don't forget the zero-padding that is needed when a word appears in some document but not others. 

Implmenet the function `get_vectors(tf,idf)`, the input is the output computed by `get_tf` and `get_idf`

(*optional challenge: implement in one line!*)

<a href="#table_of_content">back to top</a>

In [27]:
def get_vector(tf, idf):
    return np.array([tf[w]*idf[w] for w in idf])

In [28]:
# test your code
doc_vectors = [ get_vector(tf, idf) for tf in tfs ]

for v in doc_vectors:
    print(v)

[ 0.     0.     0.    ...,  0.     0.     0.169]
[ 0.     0.     0.    ...,  0.     0.     0.171]
[ 0.     0.     0.    ...,  0.     0.     0.169]
[ 0.     0.     0.    ...,  0.     0.     0.168]
[ 0.     0.     0.    ...,  0.     0.     0.167]
[ 0.769  1.529  0.    ...,  0.71   0.769  0.229]
[ 0.828  0.     0.    ...,  0.575  0.728  0.217]
[ 0.711  0.     0.835 ...,  0.561  0.711  0.125]
[ 1.097  0.     0.    ...,  0.519  0.748  0.131]
[ 0.398  0.     0.    ...,  0.579  0.835  0.147]
[ 0.  0.  0. ...,  0.  0.  0.]
[ 0.  0.  0. ...,  0.  0.  0.]
[ 0.  0.  0. ...,  0.  0.  0.]
[ 0.     0.     0.    ...,  0.337  0.     0.   ]
[ 0.     0.     0.    ...,  0.316  0.     0.   ]


<a id ="similarity"></a>

## 6. Similarity

Given two word-vectors $\vec u$ (or $u_i$) and $\vec v$ (or $v_i$), corresponding to two documents, we want to compute different similarity metrics. 

1. Cosine similarity, defined by 
$$
{\rm Sim}_{\cos} = \frac{\vec u \cdot \vec v}{|\vec u| |\vec v|}
$$

2. Jaccard similarity, defined by
$$
{\rm Sim}_{\rm Jaccard} = \frac{\sum_i \min\{u_i, v_i\}}{\sum_i \max\{u_i, v_i\}}
$$

Implement the function similarity as `sim_cis(u,v)` and `sim_jac(u,v)`. Utilize the numpy functions `numpy.linalg.norm` to compute norm of a vector and `np.dot` for computing dot-products. `np.minimum` and `np.maximum` are also useful vectorized pair-wise minimum and maximum functions.

(*optional challenge: implement both functions in one line!*)

<a href="#table_of_content">back to top</a>

In [29]:
from numpy.linalg import norm

def sim_cos(u,v):
    return np.dot(u,v)/(norm(u)*norm(v))

def sim_jac(u,v):
    return np.sum(np.minimum(u,v))/np.sum(np.maximum(u,v))

In [30]:
# test your co
# compute all the pairwise similarity metrics
size = len(doc_vectors)
matrix_cos = np.zeros((size,size))
matrix_jac = np.zeros((size,size))

for i in range(size):
    for j in range(size):
        u = doc_vectors[i]
        v = doc_vectors[j]
        matrix_cos[i][j] = sim_cos(u,v)
        matrix_jac[i][j] = sim_jac(u,v)
        
print("Cosine Similarity:")
print(matrix_cos)

print()
print("Jaccard Similarity:")
print(matrix_jac)

Cosine Similarity:
[[ 1.     0.854  0.805  0.768  0.764  0.1    0.12   0.114  0.126  0.122
   0.016  0.023  0.03   0.043  0.077]
 [ 0.854  1.     0.933  0.898  0.892  0.106  0.123  0.118  0.137  0.135
   0.013  0.021  0.028  0.039  0.072]
 [ 0.805  0.933  1.     0.966  0.957  0.099  0.116  0.116  0.137  0.129
   0.014  0.022  0.027  0.038  0.07 ]
 [ 0.768  0.898  0.966  1.     0.991  0.106  0.124  0.123  0.144  0.134
   0.014  0.022  0.029  0.039  0.073]
 [ 0.764  0.892  0.957  0.991  1.     0.107  0.127  0.125  0.144  0.136
   0.015  0.022  0.031  0.04   0.075]
 [ 0.1    0.106  0.099  0.106  0.107  1.     0.741  0.663  0.485  0.434
   0.058  0.079  0.105  0.11   0.133]
 [ 0.12   0.123  0.116  0.124  0.127  0.741  1.     0.851  0.564  0.474
   0.063  0.078  0.106  0.121  0.14 ]
 [ 0.114  0.118  0.116  0.123  0.125  0.663  0.851  1.     0.584  0.478
   0.057  0.074  0.11   0.119  0.136]
 [ 0.126  0.137  0.137  0.144  0.144  0.485  0.564  0.584  1.     0.664
   0.066  0.078  0.111  0.121

### Good Job! You've finished all the exercises!

Here is an optional bonus challenge. We often need to debug other people's code to figure out what's wrong. It's particularly difficult when the code doesn't give errors but computes the wrong quantity. Can you describe why the following implementations for some of the exercises above may be wrong? Highlight the words at the bottom to reveal the solutions!

In [ ]:
def get_idf_wrong(corpus, include_log=True):
    freq = defaultdict(int)
    for c in corpus:
        for w in c:
            freq[w] += 1
        
    N = len(corpus)
    if include_log:
        return { w:log(N/freq[w]) for w in freq }
    else:
        return { w:N/freq[w] for w in freq }
    
def get_idf(corpus, include_log=True):
    N = len(corpus)
    freq = defaultdict(int)
    words = set()
    for c in corpus:
        words |= set(c)
    for w in words:
        freq[w] = sum([w in c for c in corpus])
    if include_log:
        return {w : np.log(N/freq[w]) for w in freq}
    else:
        return {w : N/freq[w] for w in freq}




def get_sentiment_wrong(txt, wordlist):
    matching_words = [ w for w in wordlist if w in txt ]
    return len(matching_words)/len(txt)

def get_sentiment(txt, wordlist):
    matching_words = [w for w in txt if w in wordlist]
    return len(matching_words)/len(txt)



def get_vectors_wrong(tf, idf):
    return np.array([ tf[w]*idf[w] for w in tf ])

def get_vector(tf, idf):
    return np.array([tf[w]*idf[w] for w in idf])

# Solutions

Drag your mouse over the white space below this cell, and you'll see details about the solutions.  Or, if it's easier, just double-click on the white space below this cell, which will reveal the cell with hidden text.  Also, please check out the file [bagofwords_solutions.py](bagofwords_solutions.py).

<font color="white">
Solution

get_idf: the defaultdict freq here computes the total number of occurences in all the document. We only want to count it once when a word appears in a document. This may lead to a document frequency larger than N, leading to a negative idf! 

get_sentiment_wrong: if a word w appears twice in the document, it won't be counted properly!

get_vectors_wrong: tf may not contain all the words in idf. We need zero padding for words that appear in idf but not in tf! 
</font> 